In [1]:
import polars as pl
import polars.selectors as cs
import re

# Using `polars` column selectors

In this notebook, we will look at using [`polars` column selectors](https://docs.pola.rs/api/python/stable/reference/selectors.html#selectors) to perform

1. Column selections,
2. Group & Aggregate, and
3. Table Reshaping

## Topic 1 - The Data - World Bank Economic Indicators

First, let's load the World Bank's [World Development Indicators](https://databank.worldbank.org/source/world-development-indicators).

#### Attempt 1

In [2]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv')
)

ComputeError: could not parse `..` as dtype `f64` at column '2011 [YR2011]' (column number 56)

The current offset in the file is 33699 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `..` to the `null_values` list.

Original error: ```remaining bytes non-empty```

#### Attempt 2

Let's use the first hint and extend the infer schema length.

In [3]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000,
            )
)

shape: (10_173, 67)
┌────────────┬────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Country    ┆ Region ┆ Series     ┆ Series    ┆ … ┆ 2019      ┆ 2020      ┆ 2021      ┆ 2022      │
│ Name       ┆ ---    ┆ Name       ┆ Code      ┆   ┆ [YR2019]  ┆ [YR2020]  ┆ [YR2021]  ┆ [YR2022]  │
│ ---        ┆ str    ┆ ---        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆        ┆ str        ┆ str       ┆   ┆ str       ┆ str       ┆ str       ┆ str       │
╞════════════╪════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ ..        ┆ ..        ┆ 10.9      ┆ ..        │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ ..        ┆ ..        ┆ 10.9      ┆ ..        │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ ..        ┆ ..        ┆ 10.9      ┆ ..        │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ ..        ┆ ..        ┆ 10.9      ┆ ..        │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ ..        ┆ ..        ┆ 10.9      ┆ ..        │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ …          ┆ …      ┆ …          ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …         │
│ null       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ null       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ null       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ Data from  ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ database:  ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ World      ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ Deve…      ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ Last       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ Updated:   ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ 10/26/2023 ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
└────────────┴────────┴────────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘

#### Attempt 3

Looks like missing data is expressed as `".."`, let's add that as the `null_value`.

In [4]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000, 
             null_values='..',
            )
)

shape: (10_173, 67)
┌────────────┬────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Country    ┆ Region ┆ Series     ┆ Series    ┆ … ┆ 2019      ┆ 2020      ┆ 2021      ┆ 2022      │
│ Name       ┆ ---    ┆ Name       ┆ Code      ┆   ┆ [YR2019]  ┆ [YR2020]  ┆ [YR2021]  ┆ [YR2022]  │
│ ---        ┆ str    ┆ ---        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆        ┆ str        ┆ str       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ …          ┆ …      ┆ …          ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …         │
│ null       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ null       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ null       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ Data from  ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ database:  ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ World      ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ Deve…      ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ Last       ┆ null   ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ Updated:   ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
│ 10/26/2023 ┆        ┆            ┆           ┆   ┆           ┆           ┆           ┆           │
└────────────┴────────┴────────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘

#### Attempt 4 - Removing rows with a null `Series Name`

Notice that there are some extra rows at the bottom of the table that don't correspond to a series name/code.  Let's remove these.

In [5]:
(WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000, 
             null_values='..',
            )
 .filter(pl.col("Series Name").is_not_null())
)

shape: (10_168, 67)
┌────────────┬────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Country    ┆ Region ┆ Series     ┆ Series    ┆ … ┆ 2019      ┆ 2020      ┆ 2021      ┆ 2022      │
│ Name       ┆ ---    ┆ Name       ┆ Code      ┆   ┆ [YR2019]  ┆ [YR2020]  ┆ [YR2021]  ┆ [YR2022]  │
│ ---        ┆ str    ┆ ---        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆        ┆ str        ┆ str       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ …          ┆ …      ┆ …          ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …         │
│ World      ┆ null   ┆ Rural      ┆ SP.RUR.TO ┆ … ┆ 3.4346e9  ┆ 3.4354e9  ┆ 3.4324e9  ┆ 3.4263e9  │
│            ┆        ┆ population ┆ TL        ┆   ┆           ┆           ┆           ┆           │
│ World      ┆ null   ┆ Urban      ┆ SP.URB.TO ┆ … ┆ 4.3063e9  ┆ 4.3837e9  ┆ 4.4539e9  ┆ 4.5231e9  │
│            ┆        ┆ population ┆ TL        ┆   ┆           ┆           ┆           ┆           │
│ World      ┆ null   ┆ CO2        ┆ EN.ATM.CO ┆ … ┆ 3.5477e7  ┆ 3.3566e7  ┆ null      ┆ null      │
│            ┆        ┆ emissions  ┆ 2E.KT     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (kt)       ┆           ┆   ┆           ┆           ┆           ┆           │
│ World      ┆ null   ┆ Adolescent ┆ SP.ADO.TF ┆ … ┆ 43.372848 ┆ 42.745881 ┆ 42.479374 ┆ null      │
│            ┆        ┆ fertility  ┆ RT        ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ rate (bir… ┆           ┆   ┆           ┆           ┆           ┆           │
│ World      ┆ null   ┆ GDP        ┆ NY.GDP.MK ┆ … ┆ 8.4720e13 ┆ 8.2117e13 ┆ 8.7064e13 ┆ 8.9746e13 │
│            ┆        ┆ (constant  ┆ TP.KD     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ 2015 US$)  ┆     

#### Attempt 5 - Cleaning up the columns names using `rename` and a `dict` comprehension

Notice that there are some extra rows at the bottom of the table that don't correspond to a series name/code.  Let's remove these.

#### Review - the `rename` method

In [6]:
(WB_dev_ind
 .rename({'1960 [YR1960]':'1960',
          '1961 [YR1961]':'1961',
         })
 ).head()

shape: (5, 67)
┌────────────┬────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ Country    ┆ Region ┆ Series     ┆ Series    ┆ … ┆ 2019      ┆ 2020      ┆ 2021      ┆ 2022      │
│ Name       ┆ ---    ┆ Name       ┆ Code      ┆   ┆ [YR2019]  ┆ [YR2020]  ┆ [YR2021]  ┆ [YR2022]  │
│ ---        ┆ str    ┆ ---        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆        ┆ str        ┆ str       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
│ Afghanista ┆ Asia   ┆ Diabetes   ┆ SH.STA.DI ┆ … ┆ null      ┆ null      ┆ 10.9      ┆ null      │
│ n          ┆        ┆ prevalence ┆ AB.ZS     ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ (% of      ┆           ┆   ┆           ┆           ┆           ┆           │
│            ┆        ┆ popu…      ┆           ┆   ┆           ┆           ┆           ┆           │
└────────────┴────────┴────────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘

#### First, do it to one ...

In [7]:
'1960 [YR1960]'.split(' [')

['1960', 'YR1960]']

In [8]:
'1960 [YR1960]'.split(' [')[0]

'1960'

#### ... then do it to all.

In [9]:
(rename_years :=
    {old: old.split(" [")[0] 
     for old in WB_dev_ind.columns 
     if '[YR' in old
    }
)

{'1960 [YR1960]': '1960',
 '1961 [YR1961]': '1961',
 '1962 [YR1962]': '1962',
 '1963 [YR1963]': '1963',
 '1964 [YR1964]': '1964',
 '1965 [YR1965]': '1965',
 '1966 [YR1966]': '1966',
 '1967 [YR1967]': '1967',
 '1968 [YR1968]': '1968',
 '1969 [YR1969]': '1969',
 '1970 [YR1970]': '1970',
 '1971 [YR1971]': '1971',
 '1972 [YR1972]': '1972',
 '1973 [YR1973]': '1973',
 '1974 [YR1974]': '1974',
 '1975 [YR1975]': '1975',
 '1976 [YR1976]': '1976',
 '1977 [YR1977]': '1977',
 '1978 [YR1978]': '1978',
 '1979 [YR1979]': '1979',
 '1980 [YR1980]': '1980',
 '1981 [YR1981]': '1981',
 '1982 [YR1982]': '1982',
 '1983 [YR1983]': '1983',
 '1984 [YR1984]': '1984',
 '1985 [YR1985]': '1985',
 '1986 [YR1986]': '1986',
 '1987 [YR1987]': '1987',
 '1988 [YR1988]': '1988',
 '1989 [YR1989]': '1989',
 '1990 [YR1990]': '1990',
 '1991 [YR1991]': '1991',
 '1992 [YR1992]': '1992',
 '1993 [YR1993]': '1993',
 '1994 [YR1994]': '1994',
 '1995 [YR1995]': '1995',
 '1996 [YR1996]': '1996',
 '1997 [YR1997]': '1997',
 '1998 [YR19

In [10]:
fixed_WB = (WB_dev_ind :=
 pl.read_csv('./data/world_bank_raw_download_F23.csv', 
             infer_schema_length=10000, 
             null_values='..',
            )
 .filter(pl.col("Series Name").is_not_null())
 .rename(rename_years)
)

fixed_WB.head()

shape: (5, 67)
┌──────────────┬────────┬─────────────────────────┬────────────────┬───┬──────┬──────┬──────┬──────┐
│ Country Name ┆ Region ┆ Series Name             ┆ Series Code    ┆ … ┆ 2019 ┆ 2020 ┆ 2021 ┆ 2022 │
│ ---          ┆ ---    ┆ ---                     ┆ ---            ┆   ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ str          ┆ str    ┆ str                     ┆ str            ┆   ┆ f64  ┆ f64  ┆ f64  ┆ f64  │
╞══════════════╪════════╪═════════════════════════╪════════════════╪═══╪══════╪══════╪══════╪══════╡
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (%  ┆ SH.STA.DIAB.ZS ┆ … ┆ null ┆ null ┆ 10.9 ┆ null │
│              ┆        ┆ of popu…                ┆                ┆   ┆      ┆      ┆      ┆      │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (%  ┆ SH.STA.DIAB.ZS ┆ … ┆ null ┆ null ┆ 10.9 ┆ null │
│              ┆        ┆ of popu…                ┆                ┆   ┆      ┆      ┆      ┆      │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (%  ┆ SH.STA.DIAB.ZS ┆ … ┆ null ┆ null ┆ 10.9 ┆ null │
│              ┆        ┆ of popu…                ┆                ┆   ┆      ┆      ┆      ┆      │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (%  ┆ SH.STA.DIAB.ZS ┆ … ┆ null ┆ null ┆ 10.9 ┆ null │
│              ┆        ┆ of popu…                ┆                ┆   ┆      ┆      ┆      ┆      │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (%  ┆ SH.STA.DIAB.ZS ┆ … ┆ null ┆ null ┆ 10.9 ┆ null │
│              ┆        ┆ of popu…                ┆                ┆   ┆      ┆      ┆      ┆      │
└──────────────┴────────┴─────────────────────────┴────────────────┴───┴──────┴──────┴──────┴──────┘

## Topic 2 - Using list comprehensions to select columns

While there is a cleaner solution (see below), we can use columns selection to illustrate using `list` comprehensions to programmically process the column names in a select.

To illustrate, we look at some familiar string verbs.
- `by_name` to select by the exact name,
- `contains` to check for a sub-string,
- `starts_with` and `ends_with` to select by prefix/suffix,
- `matches` to capture more complicated patterns with a RegEx.

#### Example - Selecting BY NAME

In [11]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if col in ("Series Name", "Series Code")
         ])
).head()

shape: (5, 2)
┌─────────────────────────────────┬────────────────┐
│ Series Name                     ┆ Series Code    │
│ ---                             ┆ ---            │
│ str                             ┆ str            │
╞═════════════════════════════════╪════════════════╡
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
└─────────────────────────────────┴────────────────┘

#### Example - Selecting the Series name that CONTAINS a substring

In [12]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if 'Series' in col
         ])
).head()


shape: (5, 2)
┌─────────────────────────────────┬────────────────┐
│ Series Name                     ┆ Series Code    │
│ ---                             ┆ ---            │
│ str                             ┆ str            │
╞═════════════════════════════════╪════════════════╡
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
└─────────────────────────────────┴────────────────┘

#### Example - Selecting that STARTSWITH a substring

In [13]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if col.startswith('199')
         ])
).head()


shape: (5, 10)
┌──────┬──────┬──────┬──────┬───┬──────┬──────┬──────┬──────┐
│ 1990 ┆ 1991 ┆ 1992 ┆ 1993 ┆ … ┆ 1996 ┆ 1997 ┆ 1998 ┆ 1999 │
│ ---  ┆ ---  ┆ ---  ┆ ---  ┆   ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ f64  ┆ f64  ┆ f64  ┆ f64  ┆   ┆ f64  ┆ f64  ┆ f64  ┆ f64  │
╞══════╪══════╪══════╪══════╪═══╪══════╪══════╪══════╪══════╡
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
└──────┴──────┴──────┴──────┴───┴──────┴──────┴──────┴──────┘

#### Example - Selecting columns that MATCH a RegEx pattern

In [14]:
early_90s = re.compile(r'^199[0-4]')
the_90s = [str(y) for y in range(1990, 2000)]
[early_90s.match(y) for y in the_90s]

[<re.Match object; span=(0, 4), match='1990'>,
 <re.Match object; span=(0, 4), match='1991'>,
 <re.Match object; span=(0, 4), match='1992'>,
 <re.Match object; span=(0, 4), match='1993'>,
 <re.Match object; span=(0, 4), match='1994'>,
 None,
 None,
 None,
 None,
 None]

In [15]:
(WB_dev_ind
 .select([col 
          for col in WB_dev_ind.columns 
          if early_90s.match(col)
         ])
).head()


shape: (5, 5)
┌──────┬──────┬──────┬──────┬──────┐
│ 1990 ┆ 1991 ┆ 1992 ┆ 1993 ┆ 1994 │
│ ---  ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ f64  ┆ f64  ┆ f64  ┆ f64  ┆ f64  │
╞══════╪══════╪══════╪══════╪══════╡
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
└──────┴──────┴──────┴──────┴──────┘

## <font color="red"> Exercise 3.1.1 </font>

Create a table the contains the diabetes prevalence for each region.  For each part, use list comprehensions where possible.

Note that you will need to

1. Reshape to move all relevant series into columns and years into rows,
2. Compute the total diabetes cases for each country/year,
3. Aggregate the numerator (total diabetes) and denominator (population) for each region/year, and
4. Compute the diabetes prevalence.

In [20]:
filtered = fixed_WB.filter(
    pl.col("Series Name").str.contains("Diabetes prevalence|Population, total")
).filter(
    pl.col("Region").is_not_null()
)

year_cols = [col for col in filtered.columns if col[:4].isdigit()]

melted = filtered.melt(
    id_vars=["Country Name", "Region", "Series Name"],
    value_vars=year_cols,
    variable_name="Year",
    value_name="Value"
).with_columns([
    pl.col("Year").cast(pl.Int64)
])

pivoted = melted.pivot(
    index=["Country Name", "Region", "Year"],
    columns="Series Name",
    values="Value",
    aggregate_function="first"
).rename({
    "Diabetes prevalence (% of population ages 20 to 79)": "Diabetes %",
    "Population, total": "Population"
})

pivoted = pivoted.filter(
    pl.col("Diabetes %").is_not_null() & pl.col("Population").is_not_null()
)

with_cases = pivoted.with_columns([
    (pl.col("Diabetes %") * pl.col("Population") / 100).alias("Diabetes Cases")
])

region_summary = with_cases.group_by(["Region", "Year"]).agg([
    pl.sum("Diabetes Cases").alias("Total Diabetes"),
    pl.sum("Population").alias("Total Population")
])

result = region_summary.with_columns([
    (pl.col("Total Diabetes") / pl.col("Total Population") * 100).alias("Regional Diabetes %")
])

result.sort(["Region", "Year"]).head(10)

C:\Users\sq9890xp\AppData\Local\Temp\ipykernel_18972\845897571.py:9: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  melted = filtered.melt(
C:\Users\sq9890xp\AppData\Local\Temp\ipykernel_18972\845897571.py:18: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  pivoted = melted.pivot(


shape: (10, 5)
┌─────────────┬──────┬────────────────┬──────────────────┬─────────────────────┐
│ Region      ┆ Year ┆ Total Diabetes ┆ Total Population ┆ Regional Diabetes % │
│ ---         ┆ ---  ┆ ---            ┆ ---              ┆ ---                 │
│ str         ┆ i64  ┆ f64            ┆ f64              ┆ f64                 │
╞═════════════╪══════╪════════════════╪══════════════════╪═════════════════════╡
│ Africa      ┆ 2000 ┆ 178030.95      ┆ 1.6278467e7      ┆ 1.093659            │
│ Africa      ┆ 2011 ┆ 6.1185e7       ┆ 1.0692e9         ┆ 5.722483            │
│ Africa      ┆ 2021 ┆ 9.8796e7       ┆ 1.3900e9         ┆ 7.107609            │
│ Asia        ┆ 2000 ┆ 1.9022e7       ┆ 1.61034924e8     ┆ 11.812417           │
│ Asia        ┆ 2011 ┆ 3.2238e8       ┆ 3.8173e9         ┆ 8.445133            │
│ Asia        ┆ 2021 ┆ 4.6531e8       ┆ 4.1743e9         ┆ 11.146895           │
│ Europe      ┆ 2011 ┆ 4.6965e7       ┆ 7.22381951e8     ┆ 6.50146             │
│ Europe      ┆ 2021 ┆ 4.6000e7       ┆ 7.31153411e8     ┆ 6.291465            │
│ Middle East ┆ 2000 ┆ 105293.416     ┆ 711442.0         ┆ 14.8                │
│ Middle East ┆ 2011 ┆ 2.7643e7       ┆ 2.24941304e8     ┆ 12.289019           │
└─────────────┴──────┴────────────────┴──────────────────┴─────────────────────┘

## Topic 3 - `polars` Column Selectors

Allow column selection based on

- with familiar string verbs,
- by name,
- index,
- type, or
- other useful helper functions like `contains`, `starts_with`, or `matches`

### Familiar string-based helpers

First, we return to these string verbs.
- `by_name` to select by the exact name,
- `contains` to check for a sub-string,
- `starts_with` and `ends_with` to select by prefix/suffix,
- `matches` to capture more complicated patterns with a RegEx.

#### Example - Selecting by name

In [21]:
(WB_dev_ind
 .select(cs.by_name("Series Name", "Series Code"))
).head()

shape: (5, 2)
┌─────────────────────────────────┬────────────────┐
│ Series Name                     ┆ Series Code    │
│ ---                             ┆ ---            │
│ str                             ┆ str            │
╞═════════════════════════════════╪════════════════╡
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
└─────────────────────────────────┴────────────────┘

#### Example - Selecting the Series name and code using `contains`

In [22]:
(WB_dev_ind
 .select(cs.contains('Series'))
).head()


shape: (5, 2)
┌─────────────────────────────────┬────────────────┐
│ Series Name                     ┆ Series Code    │
│ ---                             ┆ ---            │
│ str                             ┆ str            │
╞═════════════════════════════════╪════════════════╡
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
└─────────────────────────────────┴────────────────┘

#### Example - Selecting the 1990's using `starts_with`

In [23]:
(WB_dev_ind
 .select(cs.starts_with('199'))
).head()


shape: (5, 10)
┌──────┬──────┬──────┬──────┬───┬──────┬──────┬──────┬──────┐
│ 1990 ┆ 1991 ┆ 1992 ┆ 1993 ┆ … ┆ 1996 ┆ 1997 ┆ 1998 ┆ 1999 │
│ ---  ┆ ---  ┆ ---  ┆ ---  ┆   ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ f64  ┆ f64  ┆ f64  ┆ f64  ┆   ┆ f64  ┆ f64  ┆ f64  ┆ f64  │
╞══════╪══════╪══════╪══════╪═══╪══════╪══════╪══════╪══════╡
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
└──────┴──────┴──────┴──────┴───┴──────┴──────┴──────┴──────┘

#### Example - Selecting the first five years of 1990's using `matches`

In [25]:
(WB_dev_ind
 .select(cs.matches(r'^199[0-4]'))
).head()


shape: (5, 5)
┌──────┬──────┬──────┬──────┬──────┐
│ 1990 ┆ 1991 ┆ 1992 ┆ 1993 ┆ 1994 │
│ ---  ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ f64  ┆ f64  ┆ f64  ┆ f64  ┆ f64  │
╞══════╪══════╪══════╪══════╪══════╡
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
│ null ┆ null ┆ null ┆ null ┆ null │
└──────┴──────┴──────┴──────┴──────┘

### Selecting columns by type

One unique feature of `polars` columns selectors is the ability to select columns by type.

#### Example - Selecting by index

In [26]:
(WB_dev_ind
 .select(cs.by_index(range(1,5)))
).head()

shape: (5, 4)
┌────────┬─────────────────────────────────┬────────────────┬──────┐
│ Region ┆ Series Name                     ┆ Series Code    ┆ 1960 │
│ ---    ┆ ---                             ┆ ---            ┆ ---  │
│ str    ┆ str                             ┆ str            ┆ f64  │
╞════════╪═════════════════════════════════╪════════════════╪══════╡
│ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS ┆ null │
│ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS ┆ null │
│ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS ┆ null │
│ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS ┆ null │
│ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS ┆ null │
└────────┴─────────────────────────────────┴────────────────┴──────┘

#### Example - Selecting all string columns.

In [27]:
(WB_dev_ind
 .select(cs.by_dtype(pl.String))
).head()

shape: (5, 4)
┌──────────────┬────────┬─────────────────────────────────┬────────────────┐
│ Country Name ┆ Region ┆ Series Name                     ┆ Series Code    │
│ ---          ┆ ---    ┆ ---                             ┆ ---            │
│ str          ┆ str    ┆ str                             ┆ str            │
╞══════════════╪════════╪═════════════════════════════════╪════════════════╡
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
└──────────────┴────────┴─────────────────────────────────┴────────────────┘

In [28]:
(WB_dev_ind
 .select(cs.string())
).head()

shape: (5, 4)
┌──────────────┬────────┬─────────────────────────────────┬────────────────┐
│ Country Name ┆ Region ┆ Series Name                     ┆ Series Code    │
│ ---          ┆ ---    ┆ ---                             ┆ ---            │
│ str          ┆ str    ┆ str                             ┆ str            │
╞══════════════╪════════╪═════════════════════════════════╪════════════════╡
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ SH.STA.DIAB.ZS │
└──────────────┴────────┴─────────────────────────────────┴────────────────┘

### Combining selectors with set operations

Another useful feature: Combine with set operations:

- **Complement.** `~selector1`
- **Union.** `selector1 | selector2`
- **Intersection.** `selector1 & selector2`
- **Difference.** `selector1 - selector2`
- **Symmetric difference.** `selector1 ^ selector2`

#### Example - All the string/index columns excluding the `Series Code`

In [29]:
(WB_dev_ind
 .select(cs.string() - cs.contains('Code'))
).head()


shape: (5, 3)
┌──────────────┬────────┬─────────────────────────────────┐
│ Country Name ┆ Region ┆ Series Name                     │
│ ---          ┆ ---    ┆ ---                             │
│ str          ┆ str    ┆ str                             │
╞══════════════╪════════╪═════════════════════════════════╡
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… │
└──────────────┴────────┴─────────────────────────────────┘

#### Example - All index columns (minus the `Series Code`) and the `1990`s

In [30]:
(WB_dev_ind
 .select(cs.string() - cs.contains('Code') | cs.starts_with('199'))
).head()


shape: (5, 13)
┌──────────────┬────────┬─────────────────────────────────┬──────┬───┬──────┬──────┬──────┬──────┐
│ Country Name ┆ Region ┆ Series Name                     ┆ 1990 ┆ … ┆ 1996 ┆ 1997 ┆ 1998 ┆ 1999 │
│ ---          ┆ ---    ┆ ---                             ┆ ---  ┆   ┆ ---  ┆ ---  ┆ ---  ┆ ---  │
│ str          ┆ str    ┆ str                             ┆ f64  ┆   ┆ f64  ┆ f64  ┆ f64  ┆ f64  │
╞══════════════╪════════╪═════════════════════════════════╪══════╪═══╪══════╪══════╪══════╪══════╡
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
│ Afghanistan  ┆ Asia   ┆ Diabetes prevalence (% of popu… ┆ null ┆ … ┆ null ┆ null ┆ null ┆ null │
└──────────────┴────────┴─────────────────────────────────┴──────┴───┴──────┴──────┴──────┴──────┘

### Using column selectors to reshape tables.

Another place where column selectors are useful is when reshaping tables, e.g., using `pivot` or `unpivot`.

### Example - Making a tidy subset of the data

**Goal.** Compare the urban and overall population changes across regions for each year in the `1990`s for each region.

**Task 1.** Tidy up the table by reshaping, by
1. `filter` to the measures of interest,
1. `unpivot` the years in question, and
2. `pivot` to measures into separate columns.

#### Solution - Using columns selectors in `unpivot` and `pivot`

In [31]:
WB_dev_ind['Series Name'].unique()

shape: (8,)
Series: 'Series Name' [str]
[
	"Adolescent fertility rate (bir…
	"Population, total"
	"CO2 emissions (kt)"
	"Urban population"
	"Rural population"
	"Individuals using the Internet…
	"Diabetes prevalence (% of popu…
	"GDP (constant 2015 US$)"
]

In [32]:
(pop_nums :=
 WB_dev_ind
 .select(cs.string() - cs.contains('Code') 
         | cs.starts_with('199')
        )
 .filter(pl.col('Series Name')
           .str.contains(r'^(Urban|Population)')
        )
 .unpivot(on = cs.starts_with('199'),
          index= cs.string(),
          variable_name = "Year",
          )
 .pivot(on = 'Series Name',
        values = 'value',
        index = cs.by_name('Region', 'Year'),  # Column selectors captured the new column!
        aggregate_function='sum'
       )
)

shape: (70, 4)
┌──────────────┬──────┬───────────────────┬──────────────────┐
│ Region       ┆ Year ┆ Population, total ┆ Urban population │
│ ---          ┆ ---  ┆ ---               ┆ ---              │
│ str          ┆ str  ┆ f64               ┆ f64              │
╞══════════════╪══════╪═══════════════════╪══════════════════╡
│ Asia         ┆ 1990 ┆ 1.7345e10         ┆ 5.2656e9         │
│ Europe       ┆ 1990 ┆ 4.2569e9          ┆ 2.9720e9         │
│ Africa       ┆ 1990 ┆ 3.8158e9          ┆ 1.2023e9         │
│ Oceania      ┆ 1990 ┆ 1.59420834e8      ┆ 1.1343462e8      │
│ The Americas ┆ 1990 ┆ 4.3141e9          ┆ 3.1286e9         │
│ …            ┆ …    ┆ …                 ┆ …                │
│ Africa       ┆ 1999 ┆ 4.7785e9          ┆ 1.6553e9         │
│ Oceania      ┆ 1999 ┆ 1.83646284e8      ┆ 1.26016476e8     │
│ The Americas ┆ 1999 ┆ 4.9395e9          ┆ 3.7764e9         │
│ Middle East  ┆ 1999 ┆ 1.0034e9          ┆ 6.4207557e8      │
│ null         ┆ 1999 ┆ 5.8353e10         ┆ 2.5365e10        │
└──────────────┴──────┴───────────────────┴──────────────────┘

## <font color="red"> Exercise 3.1.2 </font>

Create a table the contains the diabetes prevalence for each region.  For each part, use column selectors where possible.

Note that you will need to

1. Reshape to move all relevant series into columns and years into rows,
2. Compute the total diabetes cases for each country/year,
3. Aggregate the numerator (total diabetes) and denominator (population) for each region/year, and
4. Compute the diabetes prevalence.

In [66]:
(DiabetesPrevalence := 
    WB_dev_ind
    .select(
        cs.string() - cs.contains('Code') | cs.matches(r"^(19|20)\d{2}$")
    )
    .filter(
        pl.col("Series Name").str.contains(r"^(Population, total|Diabetes prevalence)")
    )
    .unpivot(
        on=cs.matches(r"^(19|20)\d{2}$"),
        index=cs.string(),
        variable_name="Year"
    )
    .pivot(
        on="Series Name",
        values="value",
        index=cs.by_name("Country Name", "Region", "Year"),
        aggregate_function="first"
    )
    .with_columns([
        (
            pl.col("Diabetes prevalence (% of population ages 20 to 79)") / 100
            * pl.col("Population, total")
        ).alias("Diabetes Cases")
    ])
    .group_by(["Region", "Year"])
    .agg([
        pl.sum("Diabetes Cases").alias("Total Diabetes Cases"),
        pl.sum("Population, total").alias("Total Population"),
    ])
    .with_columns([
        (
            pl.col("Total Diabetes Cases") / pl.col("Total Population") * 100
        ).round(3).alias("Diabetes Prevalence (%)")
    ])
    .filter(
        pl.col("Total Diabetes Cases") > 0
    )

) 
       

shape: (20, 5)
┌──────────────┬──────┬──────────────────────┬──────────────────┬─────────────────────────┐
│ Region       ┆ Year ┆ Total Diabetes Cases ┆ Total Population ┆ Diabetes Prevalence (%) │
│ ---          ┆ ---  ┆ ---                  ┆ ---              ┆ ---                     │
│ str          ┆ str  ┆ f64                  ┆ f64              ┆ f64                     │
╞══════════════╪══════╪══════════════════════╪══════════════════╪═════════════════════════╡
│ The Americas ┆ 2000 ┆ 1.4162e7             ┆ 8.34227219e8     ┆ 1.698                   │
│ Europe       ┆ 2021 ┆ 4.6000e7             ┆ 7.32939449e8     ┆ 6.276                   │
│ Oceania      ┆ 2000 ┆ 865585.38            ┆ 3.1073305e7      ┆ 2.786                   │
│ Europe       ┆ 2011 ┆ 4.6965e7             ┆ 7.24257301e8     ┆ 6.485                   │
│ Asia         ┆ 2021 ┆ 4.6531e8             ┆ 4.1743e9         ┆ 11.147                  │
│ …            ┆ …    ┆ …                    ┆ …                ┆ …                       │
│ Africa       ┆ 2021 ┆ 9.8796e7             ┆ 1.3900e9         ┆ 7.108                   │
│ Oceania      ┆ 2021 ┆ 4.2505e6             ┆ 4.4103611e7      ┆ 9.638                   │
│ null         ┆ 2000 ┆ 1.2023e6             ┆ 5.9198e10        ┆ 0.002                   │
│ null         ┆ 2011 ┆ 5.4596e9             ┆ 6.8712e10        ┆ 7.946                   │
│ Oceania      ┆ 2011 ┆ 2.8147e6             ┆ 3.7578296e7      ┆ 7.49                    │
└──────────────┴──────┴──────────────────────┴──────────────────┴─────────────────────────┘

## Topic 4 - Using columns selectors to group and aggregate.

We can also use column selectors when transforming columns, such as

Aggregation also benefits from columns selectors, both for
1. Performing the same calulation on multiple columns,
2. Selecting columns to `group_by`, as well as
2. Performing multiple similar `agg`regations.

### Example 1 - Computing rates per 1,000 people.

First, let's illustrate using a column selector in a mutate.

**Goal.** Compute the per 1,000 person rates for CO2 and GDP for the 2000s.

In [67]:
(per_1000_rates :=
 WB_dev_ind
 .filter(pl.col('Series Name').str.contains(r'^(CO2|GDP|^Pop)'),
         pl.col('Region').is_not_null(),
        )
 .unpivot(on = cs.starts_with('200'),
          index= cs.string() - cs.contains('Code'),
          variable_name = "Year",
          )
 .pivot(on = 'Series Name',
        values = 'value',
        index = cs.by_name('Year', 'Region'),
        aggregate_function='sum',
       )
 .with_columns(cs.starts_with('Pop').cast(pl.Int64))
 .with_columns((cs.float()/pl.col('Population, total')*1000).name.suffix(' per 1,000 people'))
)

shape: (60, 7)
┌──────┬──────────────┬────────────────┬───────────┬───────────────┬───────────────┬───────────────┐
│ Year ┆ Region       ┆ Population,    ┆ CO2       ┆ GDP (constant ┆ CO2 emissions ┆ GDP (constant │
│ ---  ┆ ---          ┆ total          ┆ emissions ┆ 2015 US$)     ┆ (kt) per      ┆ 2015 US$) per │
│ str  ┆ str          ┆ ---            ┆ (kt)      ┆ ---           ┆ 1,000 p…      ┆ 1,…           │
│      ┆              ┆ i64            ┆ ---       ┆ f64           ┆ ---           ┆ ---           │
│      ┆              ┆                ┆ f64       ┆               ┆ f64           ┆ f64           │
╞══════╪══════════════╪════════════════╪═══════════╪═══════════════╪═══════════════╪═══════════════╡
│ 2000 ┆ Asia         ┆ 20219573310    ┆ 4.2543e7  ┆ 6.0354e13     ┆ 2.104059      ┆ 2.9849e6      │
│ 2000 ┆ Europe       ┆ 4295625726     ┆ 3.5008e7  ┆ 9.1349e13     ┆ 8.149786      ┆ 2.1266e7      │
│ 2000 ┆ Africa       ┆ 4897285710     ┆ 4.7456e6  ┆ 7.3803e12     ┆ 0.969022      ┆ 1.5070e6      │
│ 2000 ┆ Oceania      ┆ 186439830      ┆ 2.2406e6  ┆ 6.0884e12     ┆ 12.018077     ┆ 3.2656e7      │
│ 2000 ┆ The Americas ┆ 5005363314     ┆ 4.5209e7  ┆ 1.0959e14     ┆ 9.032179      ┆ 2.1894e7      │
│ …    ┆ …            ┆ …              ┆ …         ┆ …             ┆ …             ┆ …             │
│ 2009 ┆ Europe       ┆ 4341768852     ┆ 3.3030e7  ┆ 1.0483e14     ┆ 7.607582      ┆ 2.4144e7      │
│ 2009 ┆ Africa       ┆ 6150365700     ┆ 6.5194e6  ┆ 1.1581e13     ┆ 1.060009      ┆ 1.8830e6      │
│ 2009 ┆ Oceania      ┆ 218028024      ┆ 2.5918e6  ┆ 8.0447e12     ┆ 11.887546     ┆ 3.6898e7      │
│ 2009 ┆ The Americas ┆ 5539400508     ┆ 4.2879e7  ┆ 1.2960e14     ┆ 7.740701      ┆ 2.3397e7      │
│ 2009 ┆ Middle East  ┆ 1286670102     ┆ 9.5753e6  ┆ 1.1689e13     ┆ 7.441938      ┆ 9.0844e6      │
└──────┴──────────────┴────────────────┴───────────┴───────────────┴───────────────┴───────────────┘

### Example 2 - Compute the regional population totals.

#### Solution 1 - Without column selectors

In [68]:
(pop_nums
 .filter(pl.col('Region').is_not_null())
 .group_by('Region', 'Year')
 .agg(pl.col('Population, total').sum(),
      pl.col('Urban population').sum(),
     )
)

shape: (60, 4)
┌──────────────┬──────┬───────────────────┬──────────────────┐
│ Region       ┆ Year ┆ Population, total ┆ Urban population │
│ ---          ┆ ---  ┆ ---               ┆ ---              │
│ str          ┆ str  ┆ f64               ┆ f64              │
╞══════════════╪══════╪═══════════════════╪══════════════════╡
│ The Americas ┆ 1996 ┆ 4.7364e9          ┆ 3.5606e9         │
│ The Americas ┆ 1998 ┆ 4.8727e9          ┆ 3.7050e9         │
│ Asia         ┆ 1998 ┆ 1.9675e10         ┆ 6.7835e9         │
│ The Americas ┆ 1994 ┆ 4.5989e9          ┆ 3.4174e9         │
│ Asia         ┆ 1995 ┆ 1.8819e10         ┆ 6.1900e9         │
│ …            ┆ …    ┆ …                 ┆ …                │
│ Africa       ┆ 1990 ┆ 3.8158e9          ┆ 1.2023e9         │
│ Middle East  ┆ 1995 ┆ 9.14412582e8      ┆ 5.69331372e8     │
│ Asia         ┆ 1999 ┆ 1.9950e10         ┆ 6.9842e9         │
│ Europe       ┆ 1990 ┆ 4.2569e9          ┆ 2.9720e9         │
│ Asia         ┆ 1996 ┆ 1.9107e10         ┆ 6.3856e9         │
└──────────────┴──────┴───────────────────┴──────────────────┘

#### Solution 2 - With column selectors

In [69]:
(pop_nums_by_region_and_year :=
 pop_nums
 .drop(cs.starts_with('C'))
 .filter(pl.col('Region').is_not_null())
 .group_by(cs.string())
 .agg(cs.float().sum(),
      # cs.float().mean(),
     )
)

shape: (60, 4)
┌──────────────┬──────┬───────────────────┬──────────────────┐
│ Region       ┆ Year ┆ Population, total ┆ Urban population │
│ ---          ┆ ---  ┆ ---               ┆ ---              │
│ str          ┆ str  ┆ f64               ┆ f64              │
╞══════════════╪══════╪═══════════════════╪══════════════════╡
│ The Americas ┆ 1991 ┆ 4.3858e9          ┆ 3.2006e9         │
│ Oceania      ┆ 1994 ┆ 1.69968666e8      ┆ 1.1927238e8      │
│ Africa       ┆ 1994 ┆ 4.2244e9          ┆ 1.3971e9         │
│ Oceania      ┆ 1991 ┆ 1.62695598e8      ┆ 1.156131e8       │
│ Middle East  ┆ 1991 ┆ 8.26120518e8      ┆ 4.9716129e8      │
│ …            ┆ …    ┆ …                 ┆ …                │
│ Africa       ┆ 1992 ┆ 4.0176e9          ┆ 1.2978e9         │
│ Europe       ┆ 1994 ┆ 4.2935e9          ┆ 3.0181e9         │
│ Oceania      ┆ 1990 ┆ 1.59420834e8      ┆ 1.1343462e8      │
│ Asia         ┆ 1991 ┆ 1.7645e10         ┆ 5.4460e9         │
│ Middle East  ┆ 1996 ┆ 9.35724876e8      ┆ 5.86793526e8     │
└──────────────┴──────┴───────────────────┴──────────────────┘

In [70]:
(pop_nums_by_region_and_year :=
 pop_nums
 .drop(cs.starts_with('C'))
 .filter(pl.col('Region').is_not_null())
 .group_by(cs.string())
 .agg(cs.float().sum().name.prefix('total_'),
      cs.float().mean().name.prefix('mean_'),
     )
)

shape: (60, 6)
┌──────────────┬──────┬───────────────────┬──────────────┬──────────────────────────┬──────────────┐
│ Region       ┆ Year ┆ total_Population, ┆ total_Urban  ┆ mean_Population, total   ┆ mean_Urban   │
│ ---          ┆ ---  ┆ total             ┆ population   ┆ ---                      ┆ population   │
│ str          ┆ str  ┆ ---               ┆ ---          ┆ f64                      ┆ ---          │
│              ┆      ┆ f64               ┆ f64          ┆                          ┆ f64          │
╞══════════════╪══════╪═══════════════════╪══════════════╪══════════════════════════╪══════════════╡
│ Asia         ┆ 1994 ┆ 1.8530e10         ┆ 5.9991e9     ┆ 1.8530e10                ┆ 5.9991e9     │
│ The Americas ┆ 1996 ┆ 4.7364e9          ┆ 3.5606e9     ┆ 4.7364e9                 ┆ 3.5606e9     │
│ The Americas ┆ 1993 ┆ 4.5290e9          ┆ 3.3455e9     ┆ 4.5290e9                 ┆ 3.3455e9     │
│ Africa       ┆ 1995 ┆ 4.3312e9          ┆ 1.4470e9     ┆ 4.3312e9                 ┆ 1.4470e9     │
│ Asia         ┆ 1992 ┆ 1.7941e10         ┆ 5.6267e9     ┆ 1.7941e10                ┆ 5.6267e9     │
│ …            ┆ …    ┆ …                 ┆ …            ┆ …                        ┆ …            │
│ Asia         ┆ 1990 ┆ 1.7345e10         ┆ 5.2656e9     ┆ 1.7345e10                ┆ 5.2656e9     │
│ Europe       ┆ 1996 ┆ 4.2984e9          ┆ 3.0306e9     ┆ 4.2984e9                 ┆ 3.0306e9     │
│ Asia         ┆ 1991 ┆ 1.7645e10         ┆ 5.4460e9     ┆ 1.7645e10                ┆ 5.4460e9     │
│ Middle East  ┆ 1998 ┆ 9.80185398e8      ┆ 6.22985688e8 ┆ 9.80185398e8             ┆ 6.22985688e8 │
│ Europe       ┆ 1992 ┆ 4.2808e9          ┆ 2.9995e9     ┆ 4.2808e9                 ┆ 2.9995e9     │
└──────────────┴──────┴───────────────────┴──────────────┴──────────────────────────┴──────────────┘

### Cleaning up multiple column transformations

Finally, we can use column selectors to perform the same computation to multiple columns simultaneously.

### Example - Converting the population totals to per 1000 people. 

#### Solution 1 - Without column selectors

In [71]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns(pl.col('Population, total')/1000, 
               pl.col('Urban population')/1000,
              )
)

ColumnNotFoundError: unable to find column "Population, total"; valid columns: ["Region", "Year", "total_Population, total", "total_Urban population", "mean_Population, total", "mean_Urban population"]

#### Solution 2 - With column selectors

In [72]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns(cs.float()/1000)
)

shape: (60, 6)
┌──────────────┬──────┬───────────────────┬─────────────┬────────────────────────┬────────────┐
│ Region       ┆ Year ┆ total_Population, ┆ total_Urban ┆ mean_Population, total ┆ mean_Urban │
│ ---          ┆ ---  ┆ total             ┆ population  ┆ ---                    ┆ population │
│ str          ┆ str  ┆ ---               ┆ ---         ┆ f64                    ┆ ---        │
│              ┆      ┆ f64               ┆ f64         ┆                        ┆ f64        │
╞══════════════╪══════╪═══════════════════╪═════════════╪════════════════════════╪════════════╡
│ Asia         ┆ 1994 ┆ 1.8530e7          ┆ 5.9991e6    ┆ 1.8530e7               ┆ 5.9991e6   │
│ The Americas ┆ 1996 ┆ 4.7364e6          ┆ 3.5606e6    ┆ 4.7364e6               ┆ 3.5606e6   │
│ The Americas ┆ 1993 ┆ 4.5290e6          ┆ 3.3455e6    ┆ 4.5290e6               ┆ 3.3455e6   │
│ Africa       ┆ 1995 ┆ 4.3312e6          ┆ 1.4470e6    ┆ 4.3312e6               ┆ 1.4470e6   │
│ Asia         ┆ 1992 ┆ 1.7941e7          ┆ 5.6267e6    ┆ 1.7941e7               ┆ 5.6267e6   │
│ …            ┆ …    ┆ …                 ┆ …           ┆ …                      ┆ …          │
│ Asia         ┆ 1990 ┆ 1.7345e7          ┆ 5.2656e6    ┆ 1.7345e7               ┆ 5.2656e6   │
│ Europe       ┆ 1996 ┆ 4.2984e6          ┆ 3.0306e6    ┆ 4.2984e6               ┆ 3.0306e6   │
│ Asia         ┆ 1991 ┆ 1.7645e7          ┆ 5.4460e6    ┆ 1.7645e7               ┆ 5.4460e6   │
│ Middle East  ┆ 1998 ┆ 980185.398        ┆ 622985.688  ┆ 980185.398             ┆ 622985.688 │
│ Europe       ┆ 1992 ┆ 4.2808e6          ┆ 2.9995e6    ┆ 4.2808e6               ┆ 2.9995e6   │
└──────────────┴──────┴───────────────────┴─────────────┴────────────────────────┴────────────┘

### Example - Standardize multiple columns

#### Solution 1 - Without column selectors

In [73]:
(pop_z_scores_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns((pl.col('Population, total') - pl.col('Population, total').mean())/pl.col('Population, total').std(), 
               (pl.col('Urban population') - pl.col('Urban population').mean())/pl.col('Urban population').std(),
              )
)

ColumnNotFoundError: unable to find column "Population, total"; valid columns: ["Region", "Year", "total_Population, total", "total_Urban population", "mean_Population, total", "mean_Urban population"]

#### Solution 2 - With columns selectors

In [74]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns((cs.float() - cs.float().mean())/cs.float().std())
)

shape: (60, 6)
┌──────────────┬──────┬───────────────────┬─────────────┬────────────────────────┬────────────┐
│ Region       ┆ Year ┆ total_Population, ┆ total_Urban ┆ mean_Population, total ┆ mean_Urban │
│ ---          ┆ ---  ┆ total             ┆ population  ┆ ---                    ┆ population │
│ str          ┆ str  ┆ ---               ┆ ---         ┆ f64                    ┆ ---        │
│              ┆      ┆ f64               ┆ f64         ┆                        ┆ f64        │
╞══════════════╪══════╪═══════════════════╪═════════════╪════════════════════════╪════════════╡
│ Asia         ┆ 1994 ┆ 2.100657          ┆ 1.721434    ┆ 2.100657               ┆ 1.721434   │
│ The Americas ┆ 1996 ┆ -0.121517         ┆ 0.539671    ┆ -0.121517              ┆ 0.539671   │
│ The Americas ┆ 1993 ┆ -0.154928         ┆ 0.435431    ┆ -0.154928              ┆ 0.435431   │
│ Africa       ┆ 1995 ┆ -0.186802         ┆ -0.48461    ┆ -0.186802              ┆ -0.48461   │
│ Asia         ┆ 1992 ┆ 2.005679          ┆ 1.540964    ┆ 2.005679               ┆ 1.540964   │
│ …            ┆ …    ┆ …                 ┆ …           ┆ …                      ┆ …          │
│ Asia         ┆ 1990 ┆ 1.9097            ┆ 1.365967    ┆ 1.9097                 ┆ 1.365967   │
│ Europe       ┆ 1996 ┆ -0.19209          ┆ 0.282847    ┆ -0.19209               ┆ 0.282847   │
│ Asia         ┆ 1991 ┆ 1.958068          ┆ 1.453374    ┆ 1.958068               ┆ 1.453374   │
│ Middle East  ┆ 1998 ┆ -0.726641         ┆ -0.883924   ┆ -0.726641              ┆ -0.883924  │
│ Europe       ┆ 1992 ┆ -0.194927         ┆ 0.267753    ┆ -0.194927              ┆ 0.267753   │
└──────────────┴──────┴───────────────────┴─────────────┴────────────────────────┴────────────┘

### Example - Standardize multiple columns (within `Region`)

#### Solution 1 - Without column selectors

In [75]:
(pop_z_scores_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns((pl.col('Population, total') - pl.col('Population, total').mean().over('Region'))/pl.col('Population, total').std().over('Region'), 
               (pl.col('Urban population') - pl.col('Urban population').mean().over('Region'))/pl.col('Urban population').std().over('Region'),
              )
)

ColumnNotFoundError: unable to find column "Population, total"; valid columns: ["Region", "Year", "total_Population, total", "total_Urban population", "mean_Population, total", "mean_Urban population"]

#### Solution 2 - With columns selectors

In [76]:
(pop_per_1K_by_region_and_year :=
 pop_nums_by_region_and_year
 .with_columns((cs.float() - cs.float().mean().over('Region'))/cs.float().std().over('Region'))
)

shape: (60, 6)
┌──────────────┬──────┬───────────────────┬─────────────┬────────────────────────┬────────────┐
│ Region       ┆ Year ┆ total_Population, ┆ total_Urban ┆ mean_Population, total ┆ mean_Urban │
│ ---          ┆ ---  ┆ total             ┆ population  ┆ ---                    ┆ population │
│ str          ┆ str  ┆ ---               ┆ ---         ┆ f64                    ┆ ---        │
│              ┆      ┆ f64               ┆ f64         ┆                        ┆ f64        │
╞══════════════╪══════╪═══════════════════╪═════════════╪════════════════════════╪════════════╡
│ Asia         ┆ 1994 ┆ -0.152371         ┆ -0.187285   ┆ -0.152371              ┆ -0.187285  │
│ The Americas ┆ 1996 ┆ 0.50228           ┆ 0.493626    ┆ 0.50228                ┆ 0.493626   │
│ The Americas ┆ 1993 ┆ -0.483487         ┆ -0.49327    ┆ -0.483487              ┆ -0.49327   │
│ Africa       ┆ 1995 ┆ 0.13916           ┆ 0.15044     ┆ 0.13916                ┆ 0.15044    │
│ Asia         ┆ 1992 ┆ -0.824406         ┆ -0.830881   ┆ -0.824406              ┆ -0.830881  │
│ …            ┆ …    ┆ …                 ┆ …           ┆ …                      ┆ …          │
│ Asia         ┆ 1990 ┆ -1.503532         ┆ -1.454959   ┆ -1.503532              ┆ -1.454959  │
│ Europe       ┆ 1996 ┆ 0.699285          ┆ 0.592759    ┆ 0.699285               ┆ 0.592759   │
│ Asia         ┆ 1991 ┆ -1.161295         ┆ -1.143246   ┆ -1.161295              ┆ -1.143246  │
│ Middle East  ┆ 1998 ┆ 1.144128          ┆ 1.148463    ┆ 1.144128               ┆ 1.148463   │
│ Europe       ┆ 1992 ┆ -0.518432         ┆ -0.701462   ┆ -0.518432              ┆ -0.701462  │
└──────────────┴──────┴───────────────────┴─────────────┴────────────────────────┴────────────┘

## <font color="red"> Exercise 3.1.3 </font>

Now redo the work in the first exercise, but also use one column selector to perform the `agg` step.

In [89]:
(DiabetesPrevalence3 :=
 WB_dev_ind
 .select(
     cs.string().exclude("Code") | cs.matches(r"^(19|20)\d{2}$")
 )
 .filter(
     pl.col("Series Name").str.contains(r"^(Population, total|Diabetes prevalence)")
 )
 .unpivot(
     on=cs.matches(r"^(19|20)\d{2}$"),
     index=cs.string(),
     variable_name="Year"
 )
 .pivot(
     values="value",
     index=cs.by_name("Country Name", "Region", "Year"),
     on="Series Name",  # FIX HERE
     aggregate_function="first"
 )
 .with_columns([
     ((pl.col("Diabetes prevalence (% of population ages 20 to 79)") / 100) * 
      pl.col("Population, total")).alias("Diabetes Cases")  # FIX HERE
 ])
 .group_by(["Region", "Year"])
 .agg(
     cs.by_name("Diabetes Cases", "Population, total").sum()
 )
 .with_columns([
     (pl.col("Diabetes Cases") / pl.col("Population, total") * 100)
     .round(2)
     .alias("Diabetes Prevalence (%)")
 ])
 .filter(pl.col("Diabetes Prevalence (%)") > 0)
)


shape: (19, 5)
┌──────────────┬──────┬────────────────┬───────────────────┬─────────────────────────┐
│ Region       ┆ Year ┆ Diabetes Cases ┆ Population, total ┆ Diabetes Prevalence (%) │
│ ---          ┆ ---  ┆ ---            ┆ ---               ┆ ---                     │
│ str          ┆ str  ┆ f64            ┆ f64               ┆ f64                     │
╞══════════════╪══════╪════════════════╪═══════════════════╪═════════════════════════╡
│ Asia         ┆ 2000 ┆ 1.9022e7       ┆ 3.3699e9          ┆ 0.56                    │
│ null         ┆ 2011 ┆ 5.4596e9       ┆ 6.8712e10         ┆ 7.95                    │
│ null         ┆ 2021 ┆ 7.5495e9       ┆ 7.7830e10         ┆ 9.7                     │
│ Europe       ┆ 2021 ┆ 4.6000e7       ┆ 7.32939449e8      ┆ 6.28                    │
│ Asia         ┆ 2011 ┆ 3.2238e8       ┆ 3.8173e9          ┆ 8.45                    │
│ …            ┆ …    ┆ …              ┆ …                 ┆ …                       │
│ Oceania      ┆ 2021 ┆ 4.2505e6       ┆ 4.4103611e7       ┆ 9.64                    │
│ The Americas ┆ 2000 ┆ 1.4162e7       ┆ 8.34227219e8      ┆ 1.7                     │
│ Middle East  ┆ 2021 ┆ 3.1208e7       ┆ 2.70106267e8      ┆ 11.55                   │
│ Middle East  ┆ 2011 ┆ 2.7643e7       ┆ 2.24941304e8      ┆ 12.29                   │
│ Asia         ┆ 2021 ┆ 4.6531e8       ┆ 4.1743e9          ┆ 11.15                   │
└──────────────┴──────┴────────────────┴───────────────────┴─────────────────────────┘